# Building Molecular Hamiltonians

## 1. Pennylane - H20 molecule

In [56]:
import numpy as np
import pennylane as qml

In [57]:
from pennylane import qchem

In [58]:
symbols = ["H", "O", "H"]
coordinates = np.array([[-0.0399, -0.0038, 0.0], [1.5780, 0.8540, 0.0], [2.7909, -0.5159, 0.0]])

Under the second quantization formalism, the electronic wavefunction is represented in the occupation number basis. The electronic Hamiltonian is given as

$H = \sum_{p,q} h_{pq} c_p^\dagger c_q +
      \frac{1}{2} \sum_{p,q,r,s} h_{pqrs} c_p^\dagger c_q^\dagger c_r c_s$

A state of M qubits can be used to encode the occupation number elements, meaning that the fermionic Hamiltonian needs to be mapped to a qubit Hamiltonian via the Jordan-Wigner transformation (into a linear combination of Pauli strings)

$H = \sum_j C_j \otimes_i \sigma_i^{(j)}$

In [59]:
molecule = qchem.Molecule(symbols, coordinates)
H, qubits = qchem.molecular_hamiltonian(molecule)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Number of qubits: 14
Qubit Hamiltonian
-46.46418145372083 * I([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]) + 12.412625771378313 * Z(0) + -0.12507007495317712 * (Y(0) @ Z(1) @ Y(2)) + -0.12507007495317712 * (X(0) @ Z(1) @ X(2)) + 6.264550059762114e-05 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Y(4)) + 6.264550059762114e-05 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ X(4)) + 0.04274502915517567 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Y(6)) + 0.04274502915517567 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ X(6)) + -0.07164205833936563 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Y(10)) + -0.07164205833936563 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ X(10)) + 0.00027347855269432484 * (Y(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ Y(12)) + 0.00027347855269432484 * (X(0) @ Z(1) @ Z(2) @ Z(3) @ Z(4) @ Z(5) @ Z(6) @ Z(7) @ Z(8) @ Z(9) @ Z(10) @ Z(11) @ X(12)) + 1.6538777936362181 * Z(2) + 0.2367114623260683 * (Z(

In [60]:
charge = 0
multiplicity = 1
basis_set = "sto-3g" # minimal basis set

In [61]:
electrons = 10
orbitals = 7
core, active = qchem.active_space(electrons, orbitals, active_electrons=4, active_orbitals=4)

In [62]:
print("List of core orbitals: {:}".format(core))
print("List of active orbitals: {:}".format(active))
print("Number of qubits: {:}".format(2 * len(active)))

List of core orbitals: [0, 1, 2]
List of active orbitals: [3, 4, 5, 6]
Number of qubits: 8


In [1]:
molecule = qchem.Molecule(
    symbols,
    coordinates,
    charge=charge,
    mult=multiplicity,
    basis_name=basis_set
)

H, qubits = qchem.molecular_hamiltonian(
    molecule,
    active_electrons=4,
    active_orbitals=4,
)

print("Number of qubits required to perform quantum simulations: {:}".format(qubits))
print("Hamiltonian of the water molecule")


NameError: name 'qchem' is not defined

## 2. VQE overview in PennyLane

### H2 molecule

In [64]:
import numpy as np
import pennylane as qml

In [65]:
#from jax import numpy as jnp
#jax.config.update("jax_platform_name", "cpu")  # set to CPU
#jax.config.update("jax_enable_x64", True)   # 64 bit precision for float operations

In [66]:
dataset = qml.data.load('qchem', molname="H2")[0]  # return first item from qchem h2 dataset
H, qubits = dataset.hamiltonian, len(dataset.hamiltonian.wires)  # define hamiltonian and number of qubits from dataset

In [67]:
# cross-reference by manually constructing hamiltonian with procedure in sec. 1

symbols = ["H", "H"]
coordinates = np.array([
    [0.0, 0.0, -0.6614],
    [0.0, 0.0, 0.6614]
    ])
molecule = qml.qchem.Molecule(symbols, coordinates)
H, qubits = qml.qchem.molecular_hamiltonian(molecule)
print("Number of qubits: {:}".format(qubits))
print("Qubit Hamiltonian")
print(H)

Number of qubits: 4
Qubit Hamiltonian
-0.04207255194743911 * I([0, 1, 2, 3]) + 0.17771358229091763 * Z(0) + 0.17771358229091766 * Z(1) + 0.17059759276836797 * (Z(0) @ Z(1)) + -0.24274501260941422 * Z(2) + 0.12293330449299356 * (Z(0) @ Z(2)) + 0.1676833885560135 * (Z(1) @ Z(2)) + 0.04475008406301993 * (Y(0) @ X(1) @ X(2) @ Y(3)) + -0.04475008406301993 * (Y(0) @ Y(1) @ X(2) @ X(3)) + -0.04475008406301993 * (X(0) @ X(1) @ Y(2) @ Y(3)) + 0.04475008406301993 * (X(0) @ Y(1) @ Y(2) @ X(3)) + -0.24274501260941428 * Z(3) + 0.1676833885560135 * (Z(0) @ Z(3)) + 0.12293330449299356 * (Z(1) @ Z(3)) + 0.1762766139418181 * (Z(2) @ Z(3))


In [68]:
dev = qml.device("lightning.qubit", wires=qubits) # simulator of qubits 

## OpenFermion fermion to qubit mapping

In [69]:
try:
    import openfermion
except ImportError:
    !pip install git+https://github.com/quantumlib/OpenFermion.git@master#egg=openfermion

from OpenFermion: The preferred way to specify the coefficient in openfermion is to provide an optional coefficient argument. If not provided, the coefficient defaults to 1. In the code below, the first method is preferred. The multiplication in the second method actually creates a copy of the term, which introduces some additional cost. All inplace operands (such as +=) modify classes whereas binary operands such as + create copies. Important caveats are that the empty tuple FermionOperator(()) and the empty string FermionOperator('') initializes identity. The empty initializer FermionOperator() initializes the zero operator.

In [70]:
from openfermion.ops import FermionOperator

term = FermionOperator(((3, 1), (1, 0)))
print(term)

my_term = FermionOperator('3^ 1')  # not preferred, additional cost due to copying
print(my_term)

1.0 [3^ 1]
1.0 [3^ 1]


In [71]:
good_way_to_initialize = FermionOperator('3^ 1', -1.7)
print(good_way_to_initialize)

bad_way_to_initialize = -1.7 * FermionOperator('3^ 1')
print(bad_way_to_initialize)

identity = FermionOperator('')
print(identity)

zero_operator = FermionOperator()
print(zero_operator)

-1.7 [3^ 1]
-1.7 [3^ 1]
1.0 []
0


In [72]:
my_operator = FermionOperator('4^ 1^ 3 9', 1. + 2.j)
print(my_operator)
print(my_operator.terms)

(1+2j) [4^ 1^ 3 9]
{((4, 1), (1, 1), (3, 0), (9, 0)): (1+2j)}


In [73]:
my_operator = FermionOperator('4^ 3^ 9 1', 1. + 2.j)  # target, type, coefficient
term_2 = FermionOperator('3^ 1', -1.7)
my_operator += term_2   # in-place operation modifies class, instead of copying! more efficient
#print('')
print(my_operator)
print(my_operator.terms)

-1.7 [3^ 1] +
(1+2j) [4^ 3^ 9 1]
{((4, 1), (3, 1), (9, 0), (1, 0)): (1+2j), ((3, 1), (1, 0)): -1.7}


In [74]:
try:
    import cirq
except ImportError:
    print("installing cirq...")
    !pip install --quiet cirq
    import cirq

    print("installed cirq.")

In [75]:
from openfermion.utils import commutator, count_qubits, hermitian_conjugated
from openfermion.transforms import normal_ordered

# Get the Hermitian conjugate of a FermionOperator, count its qubit, check if it is normal-ordered.
term_1 = FermionOperator('4^ 3 3^', 1. + 2.j)
print(hermitian_conjugated(term_1))
print(term_1.is_normal_ordered())
print(count_qubits(term_1))

# Normal order the term.
term_2 = normal_ordered(term_1)
print('')
print(term_2)
print(term_2.is_normal_ordered())

# Compute a commutator of the terms.
print('')
print(commutator(term_1, term_2))

(1-2j) [3 3^ 4]
False
5

(1+2j) [4^] +
(-1-2j) [4^ 3^ 3]
True

(-3+4j) [4^ 3 3^ 4^] +
(3-4j) [4^ 3 3^ 4^ 3^ 3] +
(-3+4j) [4^ 3^ 3 4^ 3 3^] +
(3-4j) [4^ 4^ 3 3^]
